In [42]:
import scipy.io
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.cluster import MiniBatchKMeans
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import fetch_openml
from sklearn.neighbors import KernelDensity
import scipy.ndimage
#import cv2
import warnings
warnings.filterwarnings("ignore")
mat = scipy.io.loadmat('Data_hoda_full.mat')

<strong>Reframe date (zero padding)</strong> 

In [43]:
maX_1=0
maX_2=0
t=0
labels=[]
for i in range(4000):
    t=t+1
    labels.append(mat['labels'][i][0])
    if(len(mat['Data'][i][0]) > maX_1):
        maX_1=len(mat['Data'][i][0])
    if (len(mat['Data'][i][0][0]) > maX_2):
        maX_2 = len(mat['Data'][i][0][0])
#print(labels)
new_frame_data=[]
# Calculate the position to place the smaller array in the center
for i in range(4000):
    larger_matrix = np.zeros((maX_1, maX_2))
    row_start = (maX_1 - len(mat['Data'][i][0])) // 2
    col_start = (maX_2 - len(mat['Data'][i][0][0])) // 2
    small_rows, small_cols = mat['Data'][i][0].shape
    larger_matrix[row_start:row_start+small_rows, col_start:col_start+small_cols] = mat['Data'][i][0]
    new_frame_data.append(larger_matrix)

rows, cols = 5, 5

# Calculate the dimensions of each smaller zone
zone_height = 54 // rows
zone_width = 45 // cols

# Create a list of subarrays representing the zones\
data_after_zoning=[]
t=0
new_frame_data=np.array(new_frame_data)

<strong> extract needed features (zoning and histogram methods)</strong> 


In [44]:
zone_rows = 54 // 5  # Divide into 5 rows
zone_cols = 45 // 5  # Divide into 5 columns

# Initialize the new 5x5 matrix
new_matrix = np.zeros((5, 5))
vectorized_zonal_mean_feature=[]
# Iterate over zones and calculate the average for each
for k in range(len(new_frame_data)):
    original_matrix=new_frame_data[k]
    new_matrix = np.zeros((5, 5))
    for i in range(5):
        for j in range(5):
            zone = original_matrix[i * zone_rows:(i + 1) * zone_rows, j * zone_cols:(j + 1) * zone_cols]
            zone_average = np.mean(zone)
            new_matrix[i, j] = zone_average
    vectorized_array = new_matrix.ravel()
    vectorized_zonal_mean_feature.append(vectorized_array)
# new_matrix now contains the 5x5 matrix with the average values of each zone
vectorized_zonal_mean_feature=np.array(vectorized_zonal_mean_feature)

histogram_feature_set=[]
for i in range(4000):
    col_sums = np.sum(np.abs(new_frame_data[i]), axis=0)    
    row_sums = np.sum(np.abs(new_frame_data[i]), axis=1)
    concatenated_array = np.concatenate((col_sums, row_sums), axis=0)
    histogram_feature_set.append(concatenated_array)

X_train_zoning=vectorized_zonal_mean_feature[:3000]
y_train=labels[:3000]
X_test_zoning=vectorized_zonal_mean_feature[3000:]
y_test=labels[3000:]

<strong>Functions used in kmeans clustering</strong> 


In [45]:
#######################
###k-means functions 
def infer_cluster_labels(kmeans, actual_labels):
    """
    Associates most probable label with each cluster in KMeans model
    returns: dictionary of clusters assigned to each label
    """

    inferred_labels = {}

    for i in range(kmeans.n_clusters):

         # find index of points in cluster
        labels = []
        index = np.where(kmeans.labels_ == i)
       
        #print(index)
        #print((index[0][0]))
        # append actual labels for each point in cluster
        #print(actual_labels[index[0]])
        for j in range(len(index[0])):
            labels.append(actual_labels[index[0][j]])
        #labels.append(actual_labels[index[0]])
        #print(len(labels))
        #print(labels)
        # determine most common label
        if len(labels) == 1:
            counts = np.bincount(labels)
        else:
            counts = np.bincount(np.squeeze(labels))

        # assign the cluster to a value in the inferred_labels dictionary
        if np.argmax(counts) in inferred_labels:
            # append the new number to the existing array at this slot
            inferred_labels[np.argmax(counts)].append(i)
        else:
            # create a new array in this slot
            inferred_labels[np.argmax(counts)] = [i]

       
    return inferred_labels  

def infer_data_labels(X_labels, cluster_labels):
    """
    Determines label for each array, depending on the cluster it has been assigned to.
    returns: predicted labels for each array
    """
    
    # empty array of len(X)
    predicted_labels = np.zeros(len(X_labels)).astype(np.uint8)
    
    for i, cluster in enumerate(X_labels):
        for key, value in cluster_labels.items():
            if cluster in value:
                predicted_labels[i] = key
                
    return predicted_labels
def calculate_metrics(estimator, data, labels):

    # Calculate and print metrics
    print('Number of Clusters: {}'.format(estimator.n_clusters))
    print('Inertia: {}'.format(estimator.inertia_))
    print('Homogeneity: {}'.format(metrics.homogeneity_score(labels, estimator.labels_)))

<strong>Functions used in parzen window method</strong> 


In [46]:
#############
###parzen window functions
class ParzenWindowClassifier:
    def __init__(self, bandwidth=0.5):
        self.bandwidth = bandwidth

    def fit(self, X, y):
        self.classes = np.unique(y)
        self.kde_models = {}
        X_class=[]
        for c in self.classes:
            for i in range(len(y)):
                if ( y[i] == c):
                    X_class.append(X[i])
            #X_class = X[y == c]
            kde = KernelDensity(bandwidth=self.bandwidth)
            kde.fit(X_class)
            self.kde_models[c] = kde

    def predict(self, X):
        n_samples=1000
        class_probs = np.zeros((n_samples, len(self.classes)))

        for i, c in enumerate(self.classes):
            kde = self.kde_models[c]
            log_densities = kde.score_samples(X)
            class_probs[:, i] = np.exp(log_densities)

        return np.argmax(class_probs, axis=1)
    


<strong>Train and test section for zoning feature</strong> 


In [47]:

#1-nearest-neighbor
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train_zoning, y_train)
y_pred = knn.predict(X_test_zoning)
accuracy_knn_zoning_feature = accuracy_score(y_test, y_pred)
print("Accuracy of nearest neighbor using zonal features :", accuracy_knn_zoning_feature)
print("\n")



#model = GaussianNB()  # For Gaussian Naive Bayes
model = MultinomialNB()  # For Multinomial Naive Bayes
model.fit(X_train_zoning, y_train)
y_pred = model.predict(X_test_zoning)
accuracy_bayes_zoning_feature = accuracy_score(y_test, y_pred)
print("Accuracy of bayes using zonal features:", accuracy_bayes_zoning_feature)
print("\n")


#knn with grid search
knn = KNeighborsClassifier()
param_grid = {'n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15]}
grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_zoning, y_train)
best_k = grid_search.best_params_['n_neighbors']
best_model = grid_search.best_estimator_
accuracy = best_model.score(X_test_zoning, y_test)
print("\n")
print("best k for knn found by grid search : ",best_k)
print("Accuracy of knn using zonal features:", accuracy)
print("\n")


#parzen window
parzen_classifier = ParzenWindowClassifier(bandwidth=18)
parzen_classifier.fit(X_train_zoning, y_train)
y_pred = parzen_classifier.predict(X_test_zoning)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy of parzen using zonal features:", accuracy)
print("\n")   


#k-means

# initialize and fit KMeans algorithm on training data
kmeans = MiniBatchKMeans(n_clusters = 256)
kmeans.fit(X_train_zoning)
cluster_labels = infer_cluster_labels(kmeans, y_train)

# predict labels for testing data
test_clusters = kmeans.predict(X_test_zoning)
predicted_labels = infer_data_labels(kmeans.predict(X_test_zoning), cluster_labels)
    
# calculate and print accuracy
print('Accuracy of k-means using zonal features: {}'.format(metrics.accuracy_score(y_test, predicted_labels)))









Accuracy of nearest neighbor using zonal features : 0.869


Accuracy of bayes using zonal features: 0.83




best k for knn found by grid search :  5
Accuracy of knn using zonal features: 0.893


Accuracy of parzen using zonal features: 0.854




TypeError: Cannot cast array data from dtype('float64') to dtype('int64') according to the rule 'safe'

<strong>Train and test section for histogram feature</strong> 


In [48]:
histogram_feature_set=np.array(histogram_feature_set)
X_train_histogram=histogram_feature_set[:3000]
y_train=labels[:3000]
X_test_histogram=histogram_feature_set[3000:]
y_test=labels[3000:]
#1-nearest-neighbor
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train_histogram, y_train)
y_pred = knn.predict(X_test_histogram)
accuracy_knn_zoning_feature = accuracy_score(y_test, y_pred)
print("Accuracy of nearest neighbor using histogram features:", accuracy_knn_zoning_feature)
print("\n")


#model = GaussianNB()  # For Gaussian Naive Bayes
model = MultinomialNB()  # For Multinomial Naive Bayes
model.fit(X_train_histogram, y_train)
y_pred = model.predict(X_test_histogram)
accuracy_bayes_zoning_feature = accuracy_score(y_test, y_pred)

print("Accuracy of bayes using histogram features :", accuracy_bayes_zoning_feature)
print("\n")

#knn with grid search
knn = KNeighborsClassifier()
param_grid = {'n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15]}
grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_histogram, y_train)
best_k = grid_search.best_params_['n_neighbors']
best_model = grid_search.best_estimator_
accuracy = best_model.score(X_test_histogram, y_test)
print("best k for knn found by grid search : ",best_k)
print("Accuracy of knn using histogram features:", accuracy)
print("\n")




#parzen window

parzen_classifier = ParzenWindowClassifier(bandwidth=408)
parzen_classifier.fit(X_train_histogram, y_train)
y_pred = parzen_classifier.predict(X_test_histogram)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy of parzen using histogram features:", accuracy)
print("\n")

#k-means

kmeans = MiniBatchKMeans(n_clusters = 256)
kmeans.fit(X_train_histogram)
cluster_labels = infer_cluster_labels(kmeans, y_train)

# predict labels for testing data
test_clusters = kmeans.predict(X_test_histogram)
predicted_labels = infer_data_labels(kmeans.predict(X_test_histogram), cluster_labels)
    
#calculate and print accuracy
print('Accuracy of k-means using histogram features: {}\n'.format(metrics.accuracy_score(y_test, predicted_labels)))


Accuracy of nearest neighbor using histogram features: 0.767


Accuracy of bayes using histogram features : 0.598


best k for knn found by grid search :  7
Accuracy of knn using histogram features: 0.77


Accuracy of parzen using histogram features: 0.32




TypeError: Cannot cast array data from dtype('float64') to dtype('int64') according to the rule 'safe'


<strong>adding noise to test set</strong> 


In [49]:
###################3
###add noise to test set

def add_salt_and_pepper_noise(image, noise_ratio=0.02):
    noisy_image = image.copy()
    total_pixels = image.size
    num_salt = int(total_pixels * noise_ratio)
    salt_coords = [np.random.randint(0, i - 1, num_salt) for i in image.shape]
    noisy_image[salt_coords] = 255  # Set pixels to maximum value (salt)
    
    num_pepper = int(total_pixels * noise_ratio)
    pepper_coords = [np.random.randint(0, i - 1, num_pepper) for i in image.shape]
    noisy_image[pepper_coords] = 0  # Set pixels to minimum value (pepper)
    
    return noisy_image

# Apply noise to all images in the training dataset
noisy_train_images = []
noise_ratio = 0.1  # Adjust this value as needed

for image in new_frame_data:
    noisy_image = add_salt_and_pepper_noise(image, noise_ratio)
    noisy_train_images.append(noisy_image)
##########################
### zonal feature
# Initialize the new 5x5 matrix
new_matrix = np.zeros((5, 5))
vectorized_zonal_mean_feature_noise=[]
# Iterate over zones and calculate the average for each
for k in range(len(new_frame_data)):
    original_matrix=noisy_train_images[k]
    new_matrix = np.zeros((5, 5))
    for i in range(5):
        for j in range(5):
            zone = original_matrix[i * zone_rows:(i + 1) * zone_rows, j * zone_cols:(j + 1) * zone_cols]
            zone_average = np.mean(zone)
            new_matrix[i, j] = zone_average
    vectorized_array = new_matrix.ravel()
    vectorized_zonal_mean_feature_noise.append(vectorized_array)
# new_matrix now contains the 5x5 matrix with the average values of each zone
vectorized_zonal_mean_feature_noise=np.array(vectorized_zonal_mean_feature_noise)
###########################
###histogram feature
histogram_feature_set_noise=[]
for i in range(4000):
    col_sums = np.sum(np.abs(noisy_train_images[i]), axis=0)    
    row_sums = np.sum(np.abs(noisy_train_images[i]), axis=1)
    concatenated_array = np.concatenate((col_sums, row_sums), axis=0)
    histogram_feature_set_noise.append(concatenated_array)
histogram_feature_set_noise=np.array(histogram_feature_set_noise)


X_test_zoning_noise=vectorized_zonal_mean_feature_noise[3000:]
y_test=labels[3000:]

X_test_histogram_noise=histogram_feature_set_noise[3000:]
y_test=labels[3000:]

<strong>Train and test section for zoning feature with noisy test set</strong> 


In [50]:

#1-nearest-neighbor
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train_zoning, y_train)
y_pred = knn.predict(X_test_zoning_noise)
accuracy_knn_zoning_feature = accuracy_score(y_test, y_pred)
print("Accuracy of nearest neighbor using zonal features for noisy test set :", accuracy_knn_zoning_feature)
print("\n")

model = MultinomialNB()  # For Multinomial Naive Bayes
model.fit(X_train_zoning, y_train)
y_pred = model.predict(X_test_zoning_noise)
accuracy_bayes_zoning_feature = accuracy_score(y_test, y_pred)
print("Accuracy of bayes using zonal features for noisy test set :", accuracy_bayes_zoning_feature)
print("\n")


#knn with grid search
knn = KNeighborsClassifier()
param_grid = {'n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15]}
grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_zoning, y_train)
best_k = grid_search.best_params_['n_neighbors']
best_model = grid_search.best_estimator_
accuracy = best_model.score(X_test_zoning_noise, y_test)
print("best k for knn found by grid search : ",best_k)
print("Accuracy of knn using zonal features for noisy test set :", accuracy)
print("\n")



#parzen window
parzen_classifier = ParzenWindowClassifier(bandwidth=18)
parzen_classifier.fit(X_train_zoning, y_train)
y_pred = parzen_classifier.predict(X_test_zoning_noise)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy of parzen using zonal features for noisy test set :", accuracy)
print("\n")   



#k-means

# initialize and fit KMeans algorithm on training data
kmeans = MiniBatchKMeans(n_clusters = 256)
kmeans.fit(X_train_zoning)
cluster_labels = infer_cluster_labels(kmeans, y_train)

# predict labels for testing data
test_clusters = kmeans.predict(X_test_zoning)
predicted_labels = infer_data_labels(kmeans.predict(X_test_zoning_noise), cluster_labels)
    
# calculate and print accuracy
print('Accuracy of k-means using zonal features for noisy test set : {}'.format(metrics.accuracy_score(y_test, predicted_labels)))






Accuracy of nearest neighbor using zonal features for noisy test set : 0.725


Accuracy of bayes using zonal features for noisy test set : 0.306


best k for knn found by grid search :  5
Accuracy of knn using zonal features for noisy test set : 0.8


Accuracy of parzen using zonal features for noisy test set : 0.76


Accuracy of k-means using zonal features for noisy test set : 0.771


<strong>Train and test section for histogram feature with noisy test set</strong> 


In [57]:
#1-nearest-neighbor
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train_histogram, y_train)
y_pred = knn.predict(X_test_histogram_noise)
accuracy_knn_zoning_feature = accuracy_score(y_test, y_pred)
print("Accuracy of nearest neighbor using histogram features for noisy test set :", accuracy_knn_zoning_feature)
print("\n")


model = MultinomialNB()  # For Multinomial Naive Bayes
model.fit(X_train_histogram, y_train)
y_pred = model.predict(X_test_histogram_noise)
accuracy_bayes_zoning_feature = accuracy_score(y_test, y_pred)

print("Accuracy of bayes using histogram features for noisy test set  :", accuracy_bayes_zoning_feature)
print("\n")

#knn with grid search
knn = KNeighborsClassifier()
param_grid = {'n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15]}
grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_histogram, y_train)
best_k = grid_search.best_params_['n_neighbors']
best_model = grid_search.best_estimator_
accuracy = best_model.score(X_test_histogram_noise, y_test)
print("best k for knn found by grid search : ",best_k)
print("Accuracy of knn using histogram features for noisy test set :", accuracy)
print("\n")


#parzen window

parzen_classifier = ParzenWindowClassifier(bandwidth=408)
parzen_classifier.fit(X_train_histogram, y_train)
y_pred = parzen_classifier.predict(X_test_histogram_noise)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy of parzen using histogram features for noisy test set :", accuracy)
print("\n")


#k-means

kmeans = MiniBatchKMeans(n_clusters = 256)
kmeans.fit(X_train_histogram)
cluster_labels = infer_cluster_labels(kmeans, y_train)

# predict labels for testing data
test_clusters = kmeans.predict(X_test_histogram)
predicted_labels = infer_data_labels(kmeans.predict(X_test_histogram_noise), cluster_labels)
    
#calculate and print accuracy
print('Accuracy of k-means using histogram features for noisy test set : {}\n'.format(metrics.accuracy_score(y_test, predicted_labels)))





Accuracy of nearest neighbor using histogram features for noisy test set : 0.376


Accuracy of bayes using histogram features for noisy test set  : 0.119


best k for knn found by grid search :  7
Accuracy of knn using histogram features for noisy test set : 0.47


Accuracy of parzen using histogram features for noisy test set : 0.09


Accuracy of k-means using histogram features for noisy test set : 0.417



<strong>train with different number of samples using zoning feature</strong> 


In [52]:
accuracy_1nn_zoning_different_train_num=[]
accuracy_bayes_zoning_different_train_num=[]
accuracy_knn_zoning_different_train_num=[]
accuracy_kmeans_zoning_different_train_num=[]
accuracy_parzen_zoning_different_train_num=[]

for i in range(10):
    
    knn = KNeighborsClassifier(n_neighbors=1)
    knn.fit(X_train_zoning[0:(300*(i+1))], y_train[0:(300*(i+1))])
    y_pred = knn.predict(X_test_zoning)
    accuracy_1nn_zoning_different_train_num.append(accuracy_score(y_test, y_pred))
    
    model = MultinomialNB()  # For Multinomial Naive Bayes
    model.fit(X_train_zoning[0:(300*(i+1))], y_train[0:(300*(i+1))])
    y_pred = model.predict(X_test_zoning)
    accuracy_bayes_zoning_different_train_num.append(accuracy_score(y_test, y_pred))
    
    knn = KNeighborsClassifier()
    param_grid = {'n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15]}
    grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_train_zoning[0:(300*(i+1))], y_train[0:(300*(i+1))])
    best_k = grid_search.best_params_['n_neighbors']
    best_model = grid_search.best_estimator_
    accuracy_knn_zoning_different_train_num.append(best_model.score(X_test_zoning, y_test))
    

    
    parzen_classifier = ParzenWindowClassifier(bandwidth=18)
    parzen_classifier.fit(X_train_zoning[0:(300*(i+1))], y_train[0:(300*(i+1))])
    y_pred = parzen_classifier.predict(X_test_zoning)
    accuracy_parzen_zoning_different_train_num.append(accuracy_score(y_test, y_pred))

    
    
    
    
    

print("Accuracy of nearest neighbor using zonal features :", accuracy_1nn_zoning_different_train_num)
print("\n")
print("Accuracy of bayes using zonal features :", accuracy_bayes_zoning_different_train_num)
print("\n")
print("Accuracy of knn using zonal features :", accuracy_knn_zoning_different_train_num)
print("\n")
print("Accuracy of parzen using zonal features :", accuracy_parzen_zoning_different_train_num)
print("\n")


Accuracy of nearest neighbor using zonal features : [0.817, 0.827, 0.847, 0.849, 0.858, 0.861, 0.868, 0.87, 0.87, 0.869]


Accuracy of bayes using zonal features : [0.795, 0.817, 0.829, 0.834, 0.832, 0.833, 0.839, 0.835, 0.835, 0.83]


Accuracy of knn using zonal features : [0.817, 0.836, 0.848, 0.86, 0.867, 0.872, 0.881, 0.881, 0.891, 0.893]


Accuracy of parzen using zonal features : [0.807, 0.821, 0.839, 0.848, 0.859, 0.857, 0.857, 0.852, 0.859, 0.854]




<strong>train with different number of samples using histogram feature</strong> 


In [53]:
accuracy_1nn_histogram_different_train_num=[]
accuracy_bayes_histogram_different_train_num=[]
accuracy_knn_histogram_different_train_num=[]
accuracy_kmeans_histogram_different_train_num=[]
accuracy_parzen_histogram_different_train_num=[]


for i in range(10):
    
    knn = KNeighborsClassifier(n_neighbors=1)
    knn.fit(X_train_histogram[0:(300*(i+1))], y_train[0:(300*(i+1))])
    y_pred = knn.predict(X_test_histogram)
    accuracy_1nn_histogram_different_train_num.append(accuracy_score(y_test, y_pred))
    
    model = MultinomialNB()  # For Multinomial Naive Bayes
    model.fit(X_train_histogram[0:(300*(i+1))], y_train[0:(300*(i+1))])
    y_pred = model.predict(X_test_histogram)
    accuracy_bayes_histogram_different_train_num.append(accuracy_score(y_test, y_pred))
    
    knn = KNeighborsClassifier()
    param_grid = {'n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15]}
    grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_train_histogram[0:(300*(i+1))], y_train[0:(300*(i+1))])
    best_k = grid_search.best_params_['n_neighbors']
    best_model = grid_search.best_estimator_
    accuracy_knn_histogram_different_train_num.append(best_model.score(X_test_histogram, y_test))
    

    parzen_classifier = ParzenWindowClassifier(bandwidth=408)
    parzen_classifier.fit(X_train_histogram[0:(300*(i+1))], y_train[0:(300*(i+1))])
    y_pred = parzen_classifier.predict(X_test_histogram)
    accuracy_parzen_histogram_different_train_num.append(accuracy_score(y_test, y_pred))

    
    
    
    
    

print("Accuracy of nearest neighbor using histogram features :", accuracy_1nn_histogram_different_train_num)
print("\n")
print("Accuracy of bayes using histogram features :", accuracy_bayes_histogram_different_train_num)
print("\n")
print("Accuracy of knn using histogram features :", accuracy_knn_histogram_different_train_num)
print("\n")
print("Accuracy of parzen using histogram features :", accuracy_parzen_histogram_different_train_num)
print("\n")

Accuracy of nearest neighbor using histogram features : [0.646, 0.673, 0.717, 0.731, 0.735, 0.745, 0.745, 0.748, 0.761, 0.767]


Accuracy of bayes using histogram features : [0.568, 0.585, 0.582, 0.607, 0.587, 0.593, 0.597, 0.598, 0.604, 0.598]


Accuracy of knn using histogram features : [0.646, 0.673, 0.704, 0.719, 0.734, 0.746, 0.759, 0.766, 0.761, 0.77]


Accuracy of parzen using histogram features : [0.226, 0.249, 0.292, 0.278, 0.308, 0.31, 0.322, 0.307, 0.31, 0.32]




<strong>Plot accuracy of training with different train sets</strong> 


In [54]:
fig, axs = plt.subplots(4, 2, figsize=(15, 10))
l=[300,600,900,1200,1500,1800,2100,2400,2700,3000]


axs[0,0].plot(l, accuracy_1nn_zoning_different_train_num)
axs[0,0].set_title('accuracy vs number of train samples for nearest neighbor using zoning feature ')
axs[0,0].set_xlabel('samples')
axs[0,0].set_ylabel('accuracy')


axs[0,1].plot(l, accuracy_1nn_histogram_different_train_num)
axs[0,1].set_title('accuracy vs number of train samples for nearest neighbor using histogram feature ')
axs[0,1].set_xlabel('samples')
axs[0,1].set_ylabel('accuracy')

axs[1,0].plot(l, accuracy_bayes_zoning_different_train_num)
axs[1,0].set_title('accuracy vs number of train samples for bayes using zoning feature ')
axs[1,0].set_xlabel('samples')
axs[1,0].set_ylabel('accuracy')

axs[1,1].plot(l, accuracy_bayes_histogram_different_train_num)
axs[1,1].set_title('accuracy vs number of train samples for bayes using histogram feature ')
axs[1,1].set_xlabel('samples')
axs[1,1].set_ylabel('accuracy')

axs[2,0].plot(l, accuracy_knn_zoning_different_train_num)
axs[2,0].set_title('accuracy vs number of train samples for knn using zoning feature ')
axs[2,0].set_xlabel('samples')
axs[2,0].set_ylabel('accuracy')

axs[2,1].plot(l, accuracy_knn_histogram_different_train_num)
axs[2,1].set_title('accuracy vs number of train samples for knn using histogram feature ')
axs[2,1].set_xlabel('samples')
axs[2,1].set_ylabel('accuracy')

axs[3,0].plot(l, accuracy_parzen_zoning_different_train_num)
axs[3,0].set_title('accuracy vs number of train samples for parzen using zoning feature ')
axs[3,0].set_xlabel('samples')
axs[3,0].set_ylabel('accuracy')

axs[3,1].plot(l, accuracy_parzen_histogram_different_train_num)
axs[3,1].set_title('accuracy vs number of train samples for parzen using histogram feature ')
axs[3,1].set_xlabel('samples')
axs[3,1].set_ylabel('accuracy')

plt.tight_layout()  # Adjusts subplots to prevent overlap
plt.show()